In [5]:
import numpy as np
import pandas as pd


In [6]:
!wget -O leads.csv https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv


--2025-10-25 07:19:22--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘leads.csv’

leads.csv           100%[===================>]  78.98K  --.-KB/s    in 0.07s   

2025-10-25 07:19:22 (1.08 MB/s) - ‘leads.csv’ saved [80876/80876]



In [8]:
df=pd.read_csv("leads.csv")
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [9]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [10]:
categorical=['lead_source', 'industry','employment_status', 'location']
numerical=['number_of_courses_viewed', 'annual_income','interaction_count', 'lead_score']

In [11]:
df[categorical].isnull().sum()

lead_source          128
industry             134
employment_status    100
location              63
dtype: int64

In [12]:
df[categorical]=df[categorical].fillna("NA")

In [13]:
df[categorical].isnull().sum()

lead_source          0
industry             0
employment_status    0
location             0
dtype: int64

In [14]:
df[numerical].isnull().sum()

number_of_courses_viewed      0
annual_income               181
interaction_count             0
lead_score                    0
dtype: int64

In [15]:
df[numerical]=df[numerical].fillna(0)

In [16]:
df[numerical].isnull().sum()

number_of_courses_viewed    0
annual_income               0
interaction_count           0
lead_score                  0
dtype: int64

In [36]:
df.industry.mode()

0    retail
Name: industry, dtype: object

In [70]:
df[numerical].corr()


,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


In [71]:
from sklearn.model_selection import train_test_split
df_full_train,df_test=train_test_split(df,test_size=0.2,random_state=42)
df_train,df_val=train_test_split(df_full_train,test_size=0.25,random_state=42)

In [72]:
len(df_train),len(df_test),len(df_val)
df_train.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [75]:
df_train=df_train.reset_index(drop=True)
df_val=df_val.reset_index(drop=True)
df_test=df_test.reset_index(drop=True)
df_train.columns.tolist()


['lead_source',
 'industry',
 'number_of_courses_viewed',
 'annual_income',
 'employment_status',
 'location',
 'interaction_count',
 'lead_score',
 'converted']

In [76]:

y_train = df_train['converted'].values
y_val   = df_val['converted'].values
y_test  = df_test['converted'].values

from sklearn.metrics import mutual_info_score,accuracy_score
def calculate_mi(series):
    return mutual_info_score(series, df_train.converted)
mi = df_train[categorical].apply(calculate_mi)
mi.sort_values(ascending=False).to_frame(name='MI')



,MI
lead_source,0.035396
employment_status,0.012938
industry,0.011575
location,0.004464


In [77]:

df_train = df_train.drop('converted', axis=1)
df_val = df_val.drop('converted', axis=1)
df_test = df_test.drop('converted', axis=1)

assert 'converted' not in df_train.columns
assert 'converted' not in df_val.columns
assert 'converted' not in df_test.columns

In [78]:
from sklearn.feature_extraction import DictVectorizer
dicts=df_train[categorical+numerical].to_dict(orient="records")
dv=DictVectorizer(sparse=False)
X_train=dv.fit_transform(dicts)
dicts1=df_val[categorical+numerical].to_dict(orient="records")

X_val=dv.transform(dicts1)


In [83]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(
    C=1.0,
    max_iter=1000,
    random_state=42
)
model.fit(X_train,y_train)

y_pred=model.predict(X_val)
accuracy_score(y_val, y_pred)


/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8532423208191127

np.float64(0.8532423208191127)

np.float64(0.8532423208191127)